# AutoModelizer
---

La idea de este proyecto es encontrar el mejor modelo de CNN que se adapte al dataset correspondiente, para ello usando algoritmos evolutivos. Este tipo de soluciones se conocen como neuroevoluciones

A continuación un ejemplo básico de como funcionan este tipo de algoritmos

In [1]:
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


def fitness(x):
    return x ** 2

population_size = 10
population = np.random.uniform(-10, 10, population_size)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def select_parent_tournament(population, scores, k=3):
    selection_ix = np.random.randint(len(population), size=k)
    selected = population[selection_ix]
    ix = np.argmax(scores[selection_ix])
    return selected[ix]

def crossover(p1, p2):
    child = (p1 + p2) / 2
    return child

def mutate(x):
    mutation_chance = 0.1
    if np.random.rand() < mutation_chance:
        x += np.random.uniform(-1, 1)
    return x

In [3]:
n_generations = 200

for generation in range(n_generations):
    scores = np.array([fitness(x) for x in population])
    new_population = []
    for _ in range(population_size):
        parent1 = select_parent_tournament(population, scores)
        parent2 = select_parent_tournament(population, scores)
        child = crossover(parent1, parent2)
        child = mutate(child)
        new_population.append(child)
    population = np.array(new_population)
    best_score = np.max(scores)
    print(f"Generación {generation}, x = {child} Mejor puntuación {best_score}")

best_solution = population[np.argmax(scores)]
print(f"Mejor solución: x = {best_solution}, f(x) = {fitness(best_solution)}")


Generación 0, x = -9.001734441583004 Mejor puntuación 95.16401276872769
Generación 1, x = -8.640179586999352 Mejor puntuación 85.21707341903098
Generación 2, x = -9.116521925679828 Mejor puntuación 85.21707341903098
Generación 3, x = -9.862386487780679 Mejor puntuación 97.52195724624808
Generación 4, x = -9.606629541695465 Mejor puntuación 98.35796302680987
Generación 5, x = -9.860328975381039 Mejor puntuación 98.35796302680987
Generación 6, x = -10.673578261068279 Mejor puntuación 115.26228863097212
Generación 7, x = -10.61112892199636 Mejor puntuación 113.92527289514935
Generación 8, x = -10.64235359153232 Mejor puntuación 113.25968996720087
Generación 9, x = -10.981532148251194 Mejor puntuación 120.59404832307449
Generación 10, x = -10.811942869891757 Mejor puntuación 120.59404832307449
Generación 11, x = -10.939134828661334 Mejor puntuación 136.67665899820378
Generación 12, x = -10.981532148251194 Mejor puntuación 128.50956031231783
Generación 13, x = -11.315007068415802 Mejor punt

Preparamos el dataset de prueba (MNIST)
---

---

In [4]:
## indicamos la máxima descendencia que queremos 
# uso 2 para simplemente comprobar que el algoritmo al completo funciona, este valor lo 
# indicará el usuario desde la interfaz
max_desc = 2

In [5]:
# DATOS DEL DATASET DE PRUEBA

num_channels = 1
px_h = 28
px_w = 28
batch_size = 64
num_classes = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preparar los DataLoader para los conjuntos de entrenamiento y validación
transform = transforms.Compose([
    transforms.Resize((px_h, px_w)),
    transforms.Grayscale(num_output_channels=num_channels),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)


Preparamos el dataset de prueba (CIFAR100)
---

---

In [6]:

# DATOS DEL DATASET DE PRUEBA CIFAR 100
num_channels = 3
px_h = 32
px_w = 32
batch_size = 512
num_classes = 100

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preparar los DataLoader para los conjuntos de entrenamiento y validación
transform = transforms.Compose([
    transforms.Resize((px_h, px_w)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


## GENERACIÓN DE LA RED EN BASE A VECTORES


---

comprobamos si tenemos acceso a la GPU


In [7]:
if torch.cuda.is_available():
    print("CUDA (GPU) está disponible en tu sistema.")
else:
    print("CUDA (GPU) no está disponible en tu sistema.")

CUDA (GPU) está disponible en tu sistema.


Generamos un par de arquitecturas de prueba

Construimos una función que sea capaz de crear modelos en base a vectores que representen la arquitectura de la red.
De este modo el algorimo evolutivo puede ir adaptando y cambiando la red fácilmente

In [8]:
import torch

def build_cnn_from_individual(individual):
    layers = []
    num_layers = individual['num_conv_layers']
    fully_connected = individual['fully_connected']
    dropout = individual['dropout']
    out_channels_previous_layer = num_channels # Imagen de entrada en escala de grises (1 canal para MNIST)

    for i in range(num_layers):
        out_channels = individual['filters'][i]
        kernel_size = individual['filter_sizes'][i]
        
        conv_layer = nn.Conv2d(out_channels_previous_layer, out_channels, kernel_size=kernel_size, padding=1)
        layers.append(conv_layer)
        layers.append(nn.ReLU())
        if i < 2:
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        else:
            layers.append(nn.MaxPool2d(kernel_size=2, stride=1))


        out_channels_previous_layer = out_channels


    # Temporalmente crear un modelo para calcular el tamaño de salida de las capas convolucionales
    temp_model = nn.Sequential(*layers)

    # Calcular el tamaño de salida usando un tensor dummy
    dummy_input = torch.zeros(1, num_channels, px_h, px_w)  # Tamaño de entrada para MNIST
    output_size = temp_model(dummy_input).view(-1).shape[0]

    # Ahora, sabiendo el tamaño de salida, podemos definir las capas lineales correctamente
    layers.append(nn.Flatten())

    for i in range(fully_connected):
        layers.append(nn.Linear(in_features=output_size, out_features=output_size))
        if dropout > 0:
            layers.append(nn.Dropout(0.2))
            dropout-= 1

    layers.append(nn.Linear(output_size, num_classes))  # Salida de 10 clases para MNIST
    return nn.Sequential(*layers)


Definimos la población inicial

In [9]:
import random
from tqdm import tqdm

def generate_individual(min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes, lr_min, lr_max):
    individual = {
        'num_conv_layers': random.randint(min_conv_layers, max_conv_layers),
        'filters': [],
        'filter_sizes': [],
        'learning_rate': random.uniform(lr_min, lr_max),
        'fully_connected': random.randint(0,2),
        'dropout': random.randint(0,2)
    }

    for _ in range(individual['num_conv_layers']):
        individual['filters'].append(random.randint(min_filters, max_filters))
        individual['filter_sizes'].append(random.choice(filter_sizes))
    
    # Agrega más parámetros según sea necesario, como capas completamente conectadas, etc.

    return individual

def initialize_population(pop_size, min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes, lr_min, lr_max):
    return [generate_individual(min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes, lr_min, lr_max) for _ in range(pop_size)]


### PARAMETROS PARA POSIBLES ARQUITECTURAS DE RED
---

In [10]:
## AJUSTAR SEGÚN VAYA NECESITANDO Y TIEMPO 

population_size = 5
min_conv_layers = 1
max_conv_layers = 3
min_filters = 16
max_filters = 128
filter_sizes = [3, 5]
lr_min = 0.0001
lr_max = 0.01

In [11]:


population = initialize_population(population_size, min_conv_layers, max_conv_layers, min_filters, max_filters, filter_sizes, lr_min, lr_max)

epochs = 10

population


[{'num_conv_layers': 2,
  'filters': [98, 100],
  'filter_sizes': [5, 5],
  'learning_rate': 0.006580664111244137,
  'fully_connected': 1,
  'dropout': 2},
 {'num_conv_layers': 2,
  'filters': [70, 47],
  'filter_sizes': [3, 5],
  'learning_rate': 0.001168646634845199,
  'fully_connected': 1,
  'dropout': 2},
 {'num_conv_layers': 1,
  'filters': [112],
  'filter_sizes': [3],
  'learning_rate': 0.008707459517366928,
  'fully_connected': 0,
  'dropout': 1},
 {'num_conv_layers': 1,
  'filters': [58],
  'filter_sizes': [3],
  'learning_rate': 0.009266323914872622,
  'fully_connected': 2,
  'dropout': 1},
 {'num_conv_layers': 2,
  'filters': [42, 53],
  'filter_sizes': [3, 5],
  'learning_rate': 0.009590188540084888,
  'fully_connected': 0,
  'dropout': 0}]

## ENTRENAMIENTO Y EVALUACIÓN DE LOS MODELOS
---

In [12]:
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

def evaluate_individual(individual, train_loader, val_loader, device='cuda', epochs=5):
    # Construir el modelo basado en el individuo
    model = build_cnn_from_individual(individual).to(device)
    
    # Definir el optimizador y la función de pérdida
    optimizer = torch.optim.Adam(model.parameters(), lr=individual['learning_rate'])
    criterion = nn.CrossEntropyLoss()

    # Entrenamiento
    for epoch in range(epochs):
        model.train()
        progress_bar = tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{epochs}', unit='batch')
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

            # Actualizar la barra de progreso con la última información de pérdida
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item())})
            progress_bar.update()  # Forzar la actualización de la barra de progreso
            
        progress_bar.close()  # Cerrar la barra de progreso al final de cada época

    # Evaluación
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = correct / total
    return accuracy  # Esta es la "aptitud" del individuo


## MUTACIONES Y CRUCES
---

In [13]:
import random

def mutate_individual(individual):
    """
    Mutar un individuo cambiando aleatoriamente sus hiperparámetros.
    """
    mutation_rate = 0.3  # Probabilidad de mutar cada característica
    lr_range = (0.0001, 0.01)
    filter_range = (32, 128)  # Rango para el número de filtros
    filter_size_range = (1, 7)  # Rango para el tamaño de filtro
    fully_connected_range = (0,2) #Rango para ver cuantas fully conected se añaden
    dropout_range = (0,2) # Rango para ver cuantos dropouts se añaden
    if random.random() < mutation_rate:
        # Mutar la tasa de aprendizaje
        individual['learning_rate'] = random.uniform(*lr_range)

    # Asegurarse de que hay suficientes entradas en las listas 'filters' y 'filter_sizes'
    for i in range(individual['num_conv_layers']):
        if random.random() < mutation_rate:
            # Mutar el número de filtros en la capa i
            individual['filters'][i] = random.randint(*filter_range)
        if random.random() < mutation_rate:
            # Mutar el tamaño de filtro en la capa i
            individual['filter_sizes'][i] = random.randint(*filter_size_range)

    if random.random() < mutation_rate:
        # Mutar la tasa de aprendizaje
        individual['fully_connected'] = random.uniform(*fully_connected_range)

    return individual


Y el cruce entre individuos

In [14]:
def crossover(parent1, parent2):
    """
    Realiza un cruce uniforme entre dos individuos.
    
    Args:
        parent1 (dict): El primer individuo padre.
        parent2 (dict): El segundo individuo padre.
    
    Returns:
        dict: Un nuevo individuo hijo.
    """
    child = {}
    print(f"Padre 1: {parent1} --- Padre 2: {parent2}")
    for key in parent1:
        if key == "filters" or key == "filter_sizes":
            if random.random() < 0.5:
                child["filters"] = parent1["filters"]
                child["filter_sizes"] = parent1["filter_sizes"]
            else:
                child["filters"] = parent2["filters"]
                child["filter_sizes"] = parent2["filter_sizes"]

        else:
            if key != "num_conv_layers":
                if random.random() < 0.5:
                    child[key] = parent1[key]
                else:
                    child[key] = parent2[key]
    child["num_conv_layers"] = len(child["filters"])

    return child

In [15]:
def evaluate_population(population, train_loader, val_loader, device):
    fitness_scores = []
    
    for individual in population:
        print(individual)
        fitness = evaluate_individual(individual, train_loader, val_loader, device)
        fitness_scores.append(fitness)
    
    return fitness_scores




Recopilamos lo que hemos realizado, hemos creado las posibles mutaciones sobre las arquitecturas, los posibles cruces, la evaluación de los modelos.


Queda realizar:
- Selección de reproducción: por torneo en principio para también  puede ser por torneo o ruleta
- Creación de la nueva generación usando las funciones de mutación y cruces
- Criterios de parada
- Registro de análisis

### Selección por torneo
---


In [16]:
def tournament_selection_best4(population):
    """
    Selecciona los 4 mejores individuos de la población mediante un torneo de tamaño fijo.

    Args:
        population: La lista de individuos con sus puntuaciones de fitness.

    Returns:
        Lista de los 4 mejores individuos.
    """
    winners = []
    for _ in range(2):
        candidates = random.sample(population, 2)
        winner = max(candidates, key=lambda x: x['fitness'])
        winners.append(winner)
    return winners

## ALGORITMO EVOLUTIVO
--- 


In [17]:
"""
fitness_scores = evaluate_population(population, train_loader, val_loader, device)

# Almacenamos- los individuos y sus puntuaciones en una lista de tuplas y ordenarlos
population_with_scores = list(zip(population, fitness_scores))
population_with_scores.sort(key=lambda x: x[1], reverse=True)  # Ordena de mayor a menor aptitud

# Imprimimimos los resultados
for i, (individual, score) in enumerate(population_with_scores):
    print(f"Descendecia: {i} Individuo {individual}: Pérdida: {score}")

"""

'\nfitness_scores = evaluate_population(population, train_loader, val_loader, device)\n\n# Almacenamos- los individuos y sus puntuaciones en una lista de tuplas y ordenarlos\npopulation_with_scores = list(zip(population, fitness_scores))\npopulation_with_scores.sort(key=lambda x: x[1], reverse=True)  # Ordena de mayor a menor aptitud\n\n# Imprimimimos los resultados\nfor i, (individual, score) in enumerate(population_with_scores):\n    print(f"Descendecia: {i} Individuo {individual}: Pérdida: {score}")\n\n'

In [18]:
top_models = []

def genetic_algorithm(population, train_loader, val_loader, device, max_desc):
    desc = 0
    while desc < max_desc:
        print("***************************************")
        print()
        print(f"Generation: {desc}")
        print()
        print("***************************************")
        puntuaciones_aptitud = evaluate_population(population, train_loader, val_loader, device)

        # Ordenamos individuos
        # Añadimos las puntuaciones directamente a cada diccionario de la población
        for i, puntuacion in enumerate(puntuaciones_aptitud):
            population[i]['fitness'] = puntuacion

        population.sort(key=lambda x: x['fitness'], reverse=True)

        # Preservamos el mejor individuo (elitismo)
        mejor_individuo = population[0]
        top_models.append(mejor_individuo)
        # Seleccionamos los 4 mejores (excluyendo el mejor) para el torneo
        winners = tournament_selection_best4(population[1:])
        print(f"Los 4 mejores son: \n {winners}")

        # Realizamos cruce y mutación con descendientes que reemplazan a la población restante
        for i in range(1, 2):
            descendency = mutate_individual(crossover(winners[i-1], winners[i]))
            population[-i] = descendency   # Eliminamos las peores arquitecturas

        # Nos aseguramos que mantenemos el mejor de todos
        population[0] = mejor_individuo
        desc += 1

    return population


## RESULTADOS DE LAS ARQUITECTURAS
---

In [19]:
sol = crossover(population[1], population[2])
mutate = mutate_individual(sol)
mutate

Padre 1: {'num_conv_layers': 2, 'filters': [70, 47], 'filter_sizes': [3, 5], 'learning_rate': 0.001168646634845199, 'fully_connected': 1, 'dropout': 2} --- Padre 2: {'num_conv_layers': 1, 'filters': [112], 'filter_sizes': [3], 'learning_rate': 0.008707459517366928, 'fully_connected': 0, 'dropout': 1}


{'filters': [42],
 'filter_sizes': [3],
 'learning_rate': 0.001168646634845199,
 'fully_connected': 1,
 'dropout': 1,
 'num_conv_layers': 1}

In [20]:
population_sol = genetic_algorithm(population, train_loader, val_loader, device, max_desc)

***************************************

Generation: 0

***************************************
{'num_conv_layers': 2, 'filters': [98, 100], 'filter_sizes': [5, 5], 'learning_rate': 0.006580664111244137, 'fully_connected': 1, 'dropout': 2}


Epoch 5/5: 100%|██████████| 98/98 [00:11<00:00,  8.22batch/s, training_loss=4.611]


{'num_conv_layers': 2, 'filters': [70, 47], 'filter_sizes': [3, 5], 'learning_rate': 0.001168646634845199, 'fully_connected': 1, 'dropout': 2}


Epoch 5/5: 100%|██████████| 98/98 [00:11<00:00,  8.17batch/s, training_loss=2.505]


{'num_conv_layers': 1, 'filters': [42], 'filter_sizes': [3], 'learning_rate': 0.008707459517366928, 'fully_connected': 0, 'dropout': 1}


...

{'num_conv_layers': 2, 'filters': [98, 100], 'filter_sizes': [5, 5], 'learning_rate': 0.006580664111244137, 'fully_connected': 1, 'dropout': 2, 'fitness': 0.01}


Epoch 5/5: 100%|██████████| 98/98 [00:12<00:00,  8.03batch/s, training_loss=4.617]


{'filters': [42, 53], 'filter_sizes': [3, 5], 'learning_rate': 0.009266323914872622, 'fully_connected': 2, 'dropout': 1, 'fitness': 0.0307, 'num_conv_layers': 2}


Epoch 5/5: 100%|██████████| 98/98 [00:12<00:00,  8.10batch/s, training_loss=4.613]


Los 4 mejores son: 
 [{'num_conv_layers': 1, 'filters': [58], 'filter_sizes': [3], 'learning_rate': 0.009266323914872622, 'fully_connected': 2, 'dropout': 1, 'fitness': 0.0187}, {'num_conv_layers': 1, 'filters': [42], 'filter_sizes': [3], 'learning_rate': 0.008707459517366928, 'fully_connected': 0, 'dropout': 1, 'fitness': 0.3219}]
Padre 1: {'num_conv_layers': 1, 'filters': [58], 'filter_sizes': [3], 'learning_rate': 0.009266323914872622, 'fully_connected': 2, 'dropout': 1, 'fitness': 0.0187} --- Padre 2: {'num_conv_layers': 1, 'filters': [42], 'filter_sizes': [3], 'learning_rate': 0.008707459517366928, 'fully_connected': 0, 'dropout': 1, 'fitness': 0.3219}


In [21]:
for individual in top_models:
    print(f"Individuo {individual} ")

Individuo {'num_conv_layers': 2, 'filters': [70, 47], 'filter_sizes': [3, 5], 'learning_rate': 0.001168646634845199, 'fully_connected': 1, 'dropout': 2, 'fitness': 0.3593} 
Individuo {'num_conv_layers': 2, 'filters': [70, 47], 'filter_sizes': [3, 5], 'learning_rate': 0.001168646634845199, 'fully_connected': 1, 'dropout': 2, 'fitness': 0.3593} 
